In [2]:
import pandas as pd
import numpy as np

import re
import spacy

from nltk.corpus import stopwords 
from nltk.stem.wordnet import WordNetLemmatizer
import string

import gensim
from gensim import corpora

In [12]:
c1='nyc_tripadvisor_forum_946.csv'
c2='970_1109.csv'
c3='1110_1310.csv'

df=pd.concat([pd.read_csv(c1,index_col=0),
              pd.read_csv(c2,index_col=0),
              pd.read_csv(c3,index_col=0)])

In [20]:
df.columns=['msg']
df.reset_index(inplace=True,drop=True)
df=df[df['msg'].apply(lambda s: len(s.split(' ')))>3]#.value_counts(ascending=False).sort_index()#.plot(kind='bar')
df.reset_index(inplace=True,drop=True)


In [21]:
nlp = spacy.load('en_core_web_sm')


In [22]:
corpus=' '.join(df['msg'])
corpus

'There are a few factual and helpful websites that visitors can use to get updated information prior to and during their visit to NYC and the US.\n-------------\nThe US Department of Homeland and Security has a notice page about arrival restrictions. Keep checking daily to see what updates and restrictions are made. Also check with your own airline and your own country\'s immigration restrictions during this time:\nhttps://www.dhs.gov/publication/notices-arrival-restrictions-coronavirus\nNYC Health has an information page:\nhttps://www1.nyc.gov/site/doh/health/health-topics/coronavirus.page\nIf you\'re arriving by plane, there are ways to disinfect your space on the plane before takeoff. These tips would also work on trains and buses:\nhttps://www.nytimes.com/article/airplane-flying-coronavirus-spread.html\nThe Centers for Disease Control and Prevention has some tips on preventing the spread of coronavirus:\nhttps://www.cdc.gov/coronavirus/2019-ncov/community/index.html\nWhen you\'re i

In [25]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')

# stop loss words 
stop = set(stopwords.words('english'))

# punctuation 
exclude = set(string.punctuation) 

# lemmatization
lemma = WordNetLemmatizer() 

# One function for all the steps:
def clean(doc):
    
    # convert text into lower case + split into words
    stop_free = " ".join([i for i in doc.lower().split() if i not in stop])
    
    # remove any stop words present
    punc_free = ''.join(ch for ch in stop_free if ch not in exclude)  
    
    # remove punctuations + normalize the text
    normalized = " ".join(lemma.lemmatize(word) for word in punc_free.split())  
    return normalized



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


'factual helpful website visitor use get updated information prior visit nyc u u department homeland security notice page arrival restriction keep checking daily see update restriction made also check airline country immigration restriction time httpswwwdhsgovpublicationnoticesarrivalrestrictionscoronavirus nyc health information page httpswww1nycgovsitedohhealthhealthtopicscoronaviruspage arriving plane way disinfect space plane takeoff tip would also work train bus httpswwwnytimescomarticleairplaneflyingcoronavirusspreadhtml center disease control prevention tip preventing spread coronavirus httpswwwcdcgovcoronavirus2019ncovcommunityindexhtml nyc certain sight attraction may close time threat infection deemed high keep eye website regularly httpswwwnewyorkcoukcoronavirusinnewyork interested keeping statistic active case death recovery worldometer website updated regularly httpswwwworldometersinfocoronavirus anyone else website useful traveller please add them'

In [26]:
df['clean_msg']=df['msg'].map(clean)

In [35]:
lt_lt_words=[ i for i in df['clean_msg'].apply(lambda s: s.split(' '))]

In [38]:
gensim_dict = corpora.Dictionary(lt_lt_words)


In [41]:
#Creating document term matrix, demostrating idx and associated frequency of the word
doc_term_matrix = [gensim_dict.doc2bow(i) for i in lt_lt_words]


In [42]:
Lda = gensim.models.ldamodel.LdaModel

In [44]:
ldamodel = Lda(doc_term_matrix, num_topics=10, id2word = dict_, passes=1, random_state=0, eval_every=None)


In [60]:
tt=ldamodel.print_topics()
[re.sub(r'[""\s]','',i.split('*')[1]) for i in tt[0][1].split('+')]

['place',
 'open',
 'restaurant',
 'closed',
 'year',
 'still',
 'time',
 'may',
 'would',
 'like']

In [61]:
ldamodel.print_topics(num_words=10)

[(0,
  '0.011*"place" + 0.009*"open" + 0.007*"restaurant" + 0.007*"closed" + 0.006*"year" + 0.005*"still" + 0.004*"time" + 0.004*"may" + 0.004*"would" + 0.004*"like"'),
 (1,
  '0.013*"time" + 0.012*"hotel" + 0.010*"new" + 0.010*"would" + 0.009*"city" + 0.009*"day" + 0.008*"museum" + 0.007*"island" + 0.007*"see" + 0.007*"get"'),
 (2,
  '0.012*"train" + 0.012*"subway" + 0.010*"time" + 0.010*"would" + 0.010*"one" + 0.010*"station" + 0.009*"take" + 0.007*"like" + 0.007*"get" + 0.006*"bus"'),
 (3,
  '0.009*"good" + 0.008*"great" + 0.007*"park" + 0.007*"like" + 0.007*"food" + 0.007*"place" + 0.007*"one" + 0.006*"area" + 0.006*"lot" + 0.006*"also"'),
 (4,
  '0.012*"would" + 0.009*"people" + 0.009*"time" + 0.008*"get" + 0.008*"day" + 0.007*"nyc" + 0.007*"know" + 0.006*"thanks" + 0.006*"flight" + 0.006*"u"'),
 (5,
  '0.012*"time" + 0.010*"hotel" + 0.009*"park" + 0.006*"square" + 0.006*"service" + 0.005*"get" + 0.005*"central" + 0.004*"mask" + 0.004*"inn" + 0.004*"one"'),
 (6,
  '0.017*"may" + 0